In [23]:
import pandas as pd
import requests

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

stateList = list(us_state_abbrev.keys())

URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'

r = requests.get(URL) 

with open("confirmed-jhu-timeseries.csv", "w") as fout:
    fout.write(r.content.decode('utf-8'))
    
URL = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'

r = requests.get(URL) 

with open("deaths-jhu-timeseries.csv", "w") as fout:
    fout.write(r.content.decode('utf-8'))


In [37]:
df = pd.read_csv('confirmed-jhu-timeseries.csv')
df['Province/State'] = df['Province/State'].fillna('Total')
df = df[df['Country/Region'].str.contains('US')]
confirmedDF = df[df['Province/State'].isin(stateList)]

df = pd.read_csv('deaths-jhu-timeseries.csv')
df['Province/State'] = df['Province/State'].fillna('Total')
df = df[df['Country/Region'].str.contains('US')]
deathDF = df[df['Province/State'].isin(stateList)]


for state in stateList[0:1]:
    print(confirmedDF[confirmedDF['Province/State'].str.contains(state)].to_dict('r')[0])

{'Province/State': 'Alabama', 'Country/Region': 'US', 'Lat': 32.3182, 'Long': -86.9023, '1/22/20': 0, '1/23/20': 0, '1/24/20': 0, '1/25/20': 0, '1/26/20': 0, '1/27/20': 0, '1/28/20': 0, '1/29/20': 0, '1/30/20': 0, '1/31/20': 0, '2/1/20': 0, '2/2/20': 0, '2/3/20': 0, '2/4/20': 0, '2/5/20': 0, '2/6/20': 0, '2/7/20': 0, '2/8/20': 0, '2/9/20': 0, '2/10/20': 0, '2/11/20': 0, '2/12/20': 0, '2/13/20': 0, '2/14/20': 0, '2/15/20': 0, '2/16/20': 0, '2/17/20': 0, '2/18/20': 0, '2/19/20': 0, '2/20/20': 0, '2/21/20': 0, '2/22/20': 0, '2/23/20': 0, '2/24/20': 0, '2/25/20': 0, '2/26/20': 0, '2/27/20': 0, '2/28/20': 0, '2/29/20': 0, '3/1/20': 0, '3/2/20': 0, '3/3/20': 0, '3/4/20': 0, '3/5/20': 0, '3/6/20': 0, '3/7/20': 0, '3/8/20': 0, '3/9/20': 0, '3/10/20': 0, '3/11/20': 0, '3/12/20': 0, '3/13/20': 5, '3/14/20': 6, '3/15/20': 12, '3/16/20': 29, '3/17/20': 39, '3/18/20': 46, '3/19/20': 78, '3/20/20': 83, '3/21/20': 131, '3/22/20': 138, '3/23/20': 138.0}
